<a href="https://colab.research.google.com/github/sahildawar/Car-Speed-Measure/blob/main/Car_Speed_Measure.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install torch torchvision torchaudio opencv-python
import cv2
import torch
import numpy as np
from collections import defaultdict
from google.colab import files

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 110.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 96.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 54.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 13.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 105.1 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitl

In [ ]:
!pip install ultralytics

In [ ]:
from ultralytics import YOLO

model = YOLO('yolov8s.pt')

In [ ]:
video_path = '/content/Untitled video - Made with Clipchamp.mp4'
cap = cv2.VideoCapture(video_path)

In [ ]:
fps = cap.get(cv2.CAP_PROP_FPS)
width  = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))


In [ ]:
out = cv2.VideoWriter('output_with_speed.avi',
                      cv2.VideoWriter_fourcc(*'XVID'),
                      fps,
                      (width, height))

In [ ]:
car_tracks = defaultdict(list)
prev_centroids = {}
next_id = 0
frame_no = 0
pixel_to_meter = 0.05

In [ ]:
def assign_id(new_centroid, prev_centroids, threshold=50):
    for car_id, prev_centroid in prev_centroids.items():
        dist = np.linalg.norm(np.array(new_centroid) - np.array(prev_centroid))
        if dist < threshold:
            return car_id
    return None


In [ ]:
from ultralytics.utils import ops  # Make sure this is imported

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    results = model(frame)[0]  # YOLOv8 returns a list; get the first result

    boxes = results.boxes.xyxy.cpu().numpy()        # (xmin, ymin, xmax, ymax)
    confidences = results.boxes.conf.cpu().numpy()  # confidence scores
    class_ids = results.boxes.cls.cpu().numpy()     # class indices
    current_centroids = {}

    for i, box in enumerate(boxes):
        class_id = int(class_ids[i])
        class_name = model.names[class_id]

        if class_name != 'car':
            continue

        xmin, ymin, xmax, ymax = map(int, box)
        center_x = int((xmin + xmax) / 2)
        center_y = int((ymin + ymax) / 2)
        centroid = (center_x, center_y)

        car_id = assign_id(centroid, prev_centroids)
        if car_id is None:
            car_id = next_id
            next_id += 1

        car_tracks[car_id].append((center_x, center_y, frame_no))
        current_centroids[car_id] = centroid

        speed_text = f"ID:{car_id} | Speed: -- km/h"
        if len(car_tracks[car_id]) >= 2:
            x1, y1, f1 = car_tracks[car_id][-2]
            x2, y2, f2 = car_tracks[car_id][-1]
            pixel_dist = np.sqrt((x2 - x1) ** 2 + (y2 - y1) ** 2)
            meter_dist = pixel_dist * pixel_to_meter
            time_elapsed = (f2 - f1) / fps
            if time_elapsed > 0:
                speed = (meter_dist / time_elapsed) * 3.6  # Convert to km/h
                speed_text = f"ID:{car_id} | {speed:.1f} km/h"

        cv2.rectangle(frame, (xmin, ymin), (xmax, ymax), (0, 255, 0), 2)
        cv2.putText(frame, speed_text, (xmin, ymin - 10),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 255), 2)

    prev_centroids = current_centroids.copy()
    frame_no += 1
    out.write(frame)

cap.release()
out.release()
files.download('output_with_speed.avi')



0: 384x640 9 cars, 2 trucks, 14.5ms
Speed: 2.8ms preprocess, 14.5ms inference, 7.1ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 9 cars, 2 trucks, 10.8ms
Speed: 2.4ms preprocess, 10.8ms inference, 1.3ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 9 cars, 2 trucks, 10.8ms
Speed: 2.7ms preprocess, 10.8ms inference, 1.2ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 8 cars, 2 trucks, 10.8ms
Speed: 2.4ms preprocess, 10.8ms inference, 1.2ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 9 cars, 3 trucks, 11.0ms
Speed: 2.1ms preprocess, 11.0ms inference, 1.1ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 9 cars, 2 trucks, 10.9ms
Speed: 2.0ms preprocess, 10.9ms inference, 1.2ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 8 cars, 2 trucks, 10.8ms
Speed: 2.0ms preprocess, 10.8ms inference, 1.3ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 8 cars, 2 trucks, 10.8ms
Speed: 2.0ms preprocess, 

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
from google.colab import files
files.download('output_with_speed.avi')


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>